In [1]:
# ------------------------------------------------------spacecraft landing

In [5]:
import os
import random
import tqdm
import imageio
import math
import pandas as pd
import matplotlib.pyplot as plt
import moviepy.video.io.ImageSequenceClip
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection


path=r'C:\Users\Adminstratorius\Desktop\important to me\VU\2 semestras\Python (snek)\test'

def create_dir(path):
        if not os.path.exists(path):
            os.mkdir(path)
h = 10000.0           # m aukstis virs planetos pavirsiaus            

g = -9.8          # m/s^2 menulio laisvojo kritimo pagreitis
decel = 29.4          # m/s^2 raketos deceleracijos greitis naudojant 1 varikli

v = -330.0          # m/s kritimo greitis
deltav = 1.0       # m/s (likes deltav rezervas)


#--------atmosphere parameters
T0 = 288.15 #K pradine temperatura ASL
p0 = 101325 #Pa pradinis slegis ASL
L  = -0.0065 #K/m "temperature lapse rate"
R  = 8.31447 #duju konstanta
M  = 0.0289644 #kg/mol oro moline mase
 

class Moon:
    def __init__(self, name):                  #----------------- not used...
        self.name = 'Moon'
        self.r = rad
        patches = []
        self.circle = Circle((0, 0), rad)
        patches.append(circle)
        self.p= PatchCollection(patches, alpha = 1, color="grey")
        self.ax = plt.gca()
        self.ax.add_collection(p)              #----------------- ....at all
        
class Rocket:
    def __init__(self, deltav, v, h, g, decel):
        
        #---------------------a bunch of magic BS for aerodynamics
        
         self.area = 11.75 #aaapytyksliai vertikaliai krentancio falcon 9(+ sparnu) surface area 
            
         self.Cd = 0.4
                 
        
        
        #------------------------end of magic stuff
        
         self.status = 'falling'
         self.name = 'Rocket' # to be continued
         self.suicide_burn = 0 # suicide burn degimo aukstis
         self.deltav0 = deltav
         self.deltav0Log = False
         self.deltav = deltav
         self.v = v
         self.h = h
         self.vexh = 3000 #m/s exhaust velocity
         self.m0 = 22800 #kg tuscia mase
         self.m1 = self.m0*math.exp(self.deltav/self.vexh)
         self.g0=g
         self.g = g
         self.decel = decel
         self.atmdecel = 1 #m/s^2 paaaaati pradine atmosferos decelecarijos verte(pakeiciama iskarto)
            
            
         self.SBh =0
         self.burnStart = False   
         self.Btime = 0      # Burn time
         self.terminalv = -20 #speed of terminal descent
         self.terminal= False
         self.terminalvfinal=-5 #speed of final terminal guidance
         self.terminalheight = 25#aukstis kuriame pradedama galutinai mazinti greiti
         self.approach = 10
         self.Finalv = 0     # fiksuojamas greitis kontakto su zeme metu
         self.fatalv = -6     #m/s kaip greitai turi trenktis raketa, kad uzsilenktu
         self.crashtime=0    # time of death
         self.crash = False  # has vehicle crashed?
         self.landed = False    # has vehicle landed?
        
         self.n = 0   
         self.speed = self.v   
         self.histT = []     #saugoma istorija
         self.histV = []
         self.histH = []
            
            
    
            
    def fall(self, dt, t):
        def drag(self):
            T=T0+L*self.h   #temperatyura nuo aukscio
            p=p0*pow(1+(L*self.h)/T0,((-g)*M)/-R*L) #slegis nuo aukscio
            rho=(p*M)/(R*T) #tankio apskaiciavimas
            self.m1 =self.m0*math.exp(self.deltav/self.vexh)
            Fdrag= (rho/2) * self.area*self.Cd*math.pow(self.v, 2)
            if(self.v<-100)and(self.landed==False)and(self.crash==False):
                self.n = random.randint(-1000,1000)/1000 #gauso triuksmas
            elif(self.v<-20)and(self.landed==False)and(self.crash==False):
                self.n = random.randint(-100,100)/1000 #gauso triuksmas
            else:
                self.n=0
            self.atmdecel=Fdrag/self.m1+self.n
        
        def burn(self, dt): # logika pagal kuria veikia raketos varikliu valydmas, suletet iki -2 m/s ir palaikyti si greiti
            if(self.deltav>0): 
                if(self.h<=self.terminalheight)and(self.terminal==False):
                    self.terminalv=self.terminalvfinal
                    self.terminal = True
                if((self.v+(dt*(self.decel+self.g))>(self.terminalv))):
                    self.deltav = self.deltav -(-1*self.v -(-1*self.terminalv))
                    self.v = self.terminalv
                    self.Btime += dt
                    self.status='status: terminal guidance'  
                elif(-1*(self.v)>(dt*(self.decel + self.g)))and(self.v<self.terminalv):
                    self.deltav = self.deltav - (self.decel*dt)
                    self.v = self.v+(dt*(self.decel))
                    self.Btime += dt
                    self.status='status: retrograde burn'
            
                elif(-1*(self.v)<=self.terminalv):
                    self.deltav = self.deltav+(self.g0*dt)
                    self.v=self.terminalv
                    self.Btime += dt
                    self.status='status: terminal guidance'
            
                    
        def touchdown(self):
            if(self.h<=0.5) and not(self.crash==True):     #assumption, landing legs yra issikisusios = earlier touchdown
                self.status='status: touchdown'
                self.landed = True
                self.v = 0
                self.h=0
                if(self.deltav0Log==False):
                    self.deltav0Log=True
                    self.deltav0 -= self.deltav
        drag(self)
        if(self.h>0):
            self.suicide_burn = ((self.v * self.v)/(2*(self.decel + self.g+(self.atmdecel-self.n)*0.45)))#-(((self.v)*dt)/2)#efektyvu iki ~ 600m/s v0
        if(self.v>=(self.terminalv))and(self.deltav>0)and not(self.h<=0):     #kritimas greiteja, nebent pasiekta  "terminal guidance"
            self.v =(self.terminalv)
            self.deltav = self.deltav+(self.g0*dt)    # not in burn function, but it works like burn function
            
        elif(self.v<(self.terminalv))and not(self.h<=0):
            self.v += ((self.atmdecel+self.g)*dt)
        if(self.h>0)and not(self.g==0):#nors techniskai jo turetu reikti tik viena karta, jis periodiskai perskaiciuojamas
            
            if(self.h<=self.suicide_burn)or(self.burnStart==True)and not(self.h<0.5): #salygos kada daryti degima
                burn(self, dt)
                if (self.burnStart==False):
                    self.burnStart=True
                    self.SBh=self.h
        if(self.h<=0.5)and(self.v<(self.fatalv))or(self.crash==True):          #jei pavirsius pasiekiamas per greitai, kaboom
            self.h=0
            if(self.crash==False):
                self.Finalv = self.v
                self.crashtime = t
            self.v=0
            self.g=0
            self.crash=True
            self.status='status: R.U.D. (kaboom)' 
        if(self.h<=0.5)and not(self.crash==True):
            touchdown(self)                     #pasiekus reikiama auksti isijungia nusileidimo funkcija
        elif(self.burnStart==False)and not(self.crash==True):
            self.status='status: falling'

        if(self.deltav<=0)and(self.crash==False): #pasibaigus kurui deceleracija tampa 0
            
            self.deltav=0
            self.status='status: out of fuel(falling)'
        
        
        self.speed = -1*self.v   #nes plotinimui reikalinga teigiama verte, o v yra neigiamas vektorius
        self.histT.append(t)
        self.histV.append(self.speed)
        self.histH.append(self.h)
        self.h += (self.v*dt)+(((self.g+self.atmdecel)*(dt*dt ))/2) #keiciamas aukstis
                
class World: 
    def __init__(self, t, dt, path):
        self.t = t           # [s]
        self.dt = dt         # [s] 
        self.step = 0
        self.steps = 0
        self.objs = []
        create_dir(path)
        self.filenames = []

    def run(self, steps):
        self.steps = steps
        for self.step in tqdm.tqdm(range(1, steps + 1)):
            self.t += self.dt
            
            vehicle.fall(dt=self.dt, t=self.t)
            self.show()
            
    
    def show(self): 
        
        filename = f'{path}/world_{self.step:04d}.png'
        self.filenames.append(filename)
        
        fig= plt.figure()
        fig.subplots_adjust(wspace=0.4)
        ax1 = fig.add_subplot(221)
        
        
        ax1.axhline(0, lw=1, c='grey', label='Surface')
        ax1.scatter(0, vehicle.h, s=5)
        ax1.text(x=0, y=vehicle.h, s='%1.1f m/s' %(vehicle.speed))
        #ax1.text(x=0, y=vehicle.h, s='%1.1f m/s' %(vehicle.atmdecel))
        if(vehicle.crash==False):   #priklausomai nuo to ar vehiclas dar "gyvas" naudoja skirtinga title'a
            ax1.set_title('landing: time: %1.1f(%1.1f) s ' %(self.t, vehicle.Btime))
            ax1.set_xlabel('altitude: %1.1f m, velocity: %1.1f m/s' %(vehicle.h, vehicle.v)) #unused space can be repurposed
            if(vehicle.h<=0):
                ax1.set_xlabel('SB start altitude %1.1f m, dv spent: %1.1f m/s' %(vehicle.SBh, vehicle.deltav0)) 
        else:
            ax1.set_title('crash: time: %1.1f s ' %(vehicle.crashtime))
            ax1.set_xlabel('altitude: %1.1f m, Final velocity: %1.1f m/s' %(vehicle.h, vehicle.Finalv))
        if(vehicle.burnStart==True):
            ax1.axhline(vehicle.SBh, lw=1, c='grey', label='Surface')
        ax1.set_ylabel("altitude (m)")
        
        
        ax1.set_yticks([100, 2000, 4000, 6000, 8000, 10000])
        
        ax1.set_ylim(0, 10000)
        ax1.set_xticks([])        #panaikina x asies tick'us nes simuliacija jos nenaudoja

        ax1.ticklabel_format(style='plain')
        ax1.grid(b=True, axis='y', which='major', color='#666666', linestyle='-')
        
#----------------------------------------------------------------------ax3        
        ax2=fig.add_subplot(222)
        ax2.set_title("vehicle %s " %(vehicle.status))
        if(vehicle.crash==False):
            ax2.set_xlabel('deltav reserve: %1.1f m/s'%(vehicle.deltav))
        else:
            ax2.set_xlabel('deltav reserve: -')
        ax2.axhline(0, lw=1, c='grey', label='Surface')
        ax2.grid(b=True, axis='y', which='major', color='#666666', linestyle='-')
        ax2.scatter(0, vehicle.h, s=5)
        
        ax2.set_yticks([0, 20, 40, 60, 80, 100])
        ax2.set_xticks([])
        ax2.ticklabel_format(style='plain')
        ax2.set_ylim(0, 100)        
        
        ax2.scatter(0, vehicle.h, s=3)
        ax2.ticklabel_format(style='plain')
#------------------------------------------------------------- ax3       
        ax3=fig.add_subplot(223)
        ax3.grid(b=True, axis='y', which='major', color='#666666', linestyle='-')
        ax3.ticklabel_format(style='plain')
        
        ax3.set_xlim(0, 60)
        ax3.set_xlabel("time (s)")
        
        ax3.set_ylim(0, 10000)
        ax3.set_ylabel("altitude (m)")
        
        ax3.plot(vehicle.histT, vehicle.histH)
        ax3.fill_between(vehicle.histT, 0, vehicle.histH)
# ------------------------------------------------------------- ax4       
        ax4=fig.add_subplot(224)     
        ax4.grid(b=True, axis='y', which='major', color='#666666', linestyle='-')
        ax4.ticklabel_format(style='plain')
        
        ax4.set_xlim(0, 60)
        ax4.set_xlabel("time (s)")
        
        ax4.set_ylim(0, 450)
        ax4.set_yticks([50, 100, 200, 300, 400])
        ax4.set_ylabel("velocity (m/s)")
        
        ax4.plot(vehicle.histT, vehicle.histV)
        ax4.fill_between(vehicle.histT, 0, vehicle.histV)
        plt.savefig(filename, dpi=200)
        plt.close()
         
    def save(self): #---------------------saving mp4 here       
       image_files = [path+'/' +img for img in os.listdir(path) if img.endswith(".png")]
       clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=10)
       clip.write_videofile(f'{path}\my_video.mp4')
       for filename in set(self.filenames):
            os.remove(filename)  

vehicle = Rocket( #objektui duodami pagrindiniai parametrai
deltav = deltav,
v=v,
h=h,
g=g,
decel=decel)
#-----------------------to be continued.... or not
#planet = Moon(
#name= 'Moon'
#)

world = World(t=0.0, dt=0.1, path=path)
world.run(steps=600) 
world.save()

t:   1%|▍                                                                    | 4/601 [00:00<00:16, 36.46it/s, now=None]

Moviepy - Building video C:\Users\Adminstratorius\Desktop\important to me\VU\2 semestras\Python (snek)\test\my_video.mp4.
Moviepy - Writing video C:\Users\Adminstratorius\Desktop\important to me\VU\2 semestras\Python (snek)\test\my_video.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\Adminstratorius\Desktop\important to me\VU\2 semestras\Python (snek)\test\my_video.mp4


In [ ]:
import tqdm
import numpy as nm
import imageio
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection

plt.figure(figsize=(8, 8))
plt.scatter(0, 6300+500)

plt.axvline(0)

plt.xlim(-7300, 7300)
plt.ylim(-7300, 7300)

patches = []
circle = Circle((0, 0), 6370)
patches.append(circle)
p = PatchCollection(patches, alpha = 0.4)

ax = plt.gca()

ax.add_collection(p)
plt.axhline(6370, c='r', lw=0.5)


In [ ]:
def save(self): #---------------------saving gif here       
        with imageio.get_writer(f'{path}\LandingSequence.gif', mode='I') as writer:
            for filename in self.filenames:
                image = imageio.imread(filename)
                writer.append_data(image)
        for filename in set(self.filenames):
            os.remove(filename)  

In [ ]:
image_folder='folder_with_images'
fps=1

image_files = [image_folder+'/'+img for img in os.listdir(image_folder) if img.endswith(".png")]
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile('my_video.mp4')

In [ ]:
image_files = [path+'/' +img for img in os.listdir(path) if img.endswith(".png")]
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=10)
clip.write_videofile('my_video.mp4')